***GENERATED CODE FOR 1309211 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('OrderDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Carrier', 'transformation_label': 'String Indexer'}], 'feature': 'Carrier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '43022', 'mean': '', 'stddev': '', 'min': 'shopify', 'max': 'usps', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Carrier'}, {'feature_label': 'Carrier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Carrier')
        transformationDF = transformationDF.drop('LineItemId')
        transformationDF = transformationDF.drop('OrderId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductName', 'transformation_label': 'String Indexer'}], 'feature': 'ProductName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '43022', 'mean': '', 'stddev': '', 'min': '"Cypher Climber\'s Tape 1.5""X15 YDS"', 'max': 'stone guard', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductName'}, {'feature_label': 'ProductName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductName')
        transformationDF = transformationDF.drop('SKU')
        transformationDF = transformationDF.drop('Ref_OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ToZip', 'transformation_label': 'String Indexer'}], 'feature': 'ToZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '43022', 'mean': '65551.02', 'stddev': '34071.96', 'min': '01342-9746', 'max': '99686', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ToZip'}, {'feature_label': 'ToZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ToZip')
        transformationDF = transformationDF.drop('ProductId')
        transformationDF = transformationDF.drop('FromZip')
        transformationDF = transformationDF.drop('RetailerId')
        transformationDF = transformationDF.drop('FulfillmentLineItemId')
        transformationDF = transformationDF.drop('OrderLineItemId')
        transformationDF = transformationDF.drop('FulfillmentId')
        transformationDF = transformationDF.drop('CarrierPickedUpDateTime')
        transformationDF = transformationDF.drop(
            'CarrierDeliveredDateTime')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


def randomforestregressor(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestRegressor()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))
    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run 1309211Hooks.ipynb
try:
	#sourcePreExecutionHook()

	shippingdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1631535173895_ShippingCostData_130921_total.csv', 'filename': '1631535173895_ShippingCostData_130921_total.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/BiglynxUpdated/ShippingPriceData/ShippingCostData_130921_total.csv', 'viewFileName': 'ShippingCostData_130921_total.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 1309211Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(shippingdata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "OrderDate", "dateFormat": "yyyy-MM-dd HH-mm-ss", "transformation_label": "Convert to Date"}], "feature": "OrderDate", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "2020-01-07 21:17", "max": "2021-12-08 18:48", "missing": "0"}, "transformation": [{"transformation": "Convert to Date", "selectedAsDefault": 1}], "updatedLabel": "OrderDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShippingId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "43022", "mean": "3.23", "stddev": "1.27", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "ShippingId"}, {"transformationsData": [{"feature_label": "Carrier", "transformation_label": "String Indexer"}], "feature": "Carrier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "shopify", "max": "usps", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Carrier"}, {"transformationsData": [{"feature_label": "LineItemId", "transformation_label": "String Indexer"}], "feature": "LineItemId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "0097ABB5-55D7-4449-83BD-DDCA1B967943", "max": "FDEBA96B-021C-481E-8336-BC56860F4952", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LineItemId"}, {"transformationsData": [{"feature_label": "OrderId", "transformation_label": "String Indexer"}], "feature": "OrderId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "0172A4E3-3487-40C4-941C-AD32178222B7", "max": "FFBD9504-ED35-4DDD-B8E1-519BC2EE2C3F", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderId"}, {"transformationsData": [{"feature_label": "ProductName", "transformation_label": "String Indexer"}], "feature": "ProductName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "\"Cypher Climber's Tape 1.5\"\"X15 YDS\"", "max": "stone guard", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductName"}, {"transformationsData": [{"feature_label": "SKU", "transformation_label": "String Indexer"}], "feature": "SKU", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SKU"}, {"transformationsData": [{"feature_label": "Ref_OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "Ref_OrderLineItemId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "LineItemRef_10039218012321", "max": "LineItemRef_9990420463777", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ref_OrderLineItemId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Qty", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "43022", "mean": "1.34", "stddev": "1.7", "min": "1", "max": "22", "missing": "0"}, "updatedLabel": "Qty"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "43022", "mean": "6.87", "stddev": "3.74", "min": "1", "max": "15", "missing": "0"}, "updatedLabel": "Length"}, {"transformationsData": [{"feature_label": "ToZip", "transformation_label": "String Indexer"}], "feature": "ToZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "43022", "mean": "65551.02", "stddev": "34071.96", "min": "01342-9746", "max": "99686", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ToZip"}, {"transformationsData": [{"feature_label": "ProductId", "transformation_label": "String Indexer"}], "feature": "ProductId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "0879B8230A6B436A906958B23389382E", "max": "FE77A0E4FA0D46C58F36DBC2B99DD601", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FromZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "43022", "mean": "98052.0", "stddev": "0.0", "min": "98052", "max": "98052", "missing": "0"}, "updatedLabel": "FromZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetailerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "43022", "mean": "375544787.0", "stddev": "0.0", "min": "375544787", "max": "375544787", "missing": "0"}, "updatedLabel": "RetailerId"}, {"transformationsData": [{"feature_label": "FulfillmentLineItemId", "transformation_label": "String Indexer"}], "feature": "FulfillmentLineItemId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "003CE08F-049D-4A08-A911-454BCC4337E1", "max": "FFA77022-2015-4720-BC88-2849D80EB98B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentLineItemId"}, {"transformationsData": [{"feature_label": "OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "OrderLineItemId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "0097ABB5-55D7-4449-83BD-DDCA1B967943", "max": "FDEBA96B-021C-481E-8336-BC56860F4952", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"feature_label": "FulfillmentId", "transformation_label": "String Indexer"}], "feature": "FulfillmentId", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "01C2AAF8-A153-49C8-A56C-9613CC426C38", "max": "FFF8AD0B-42FE-4069-A784-0B621A72902B", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "43022", "mean": "1.33", "stddev": "1.7", "min": "1", "max": "22", "missing": "0"}, "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weight", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43022", "mean": "7.94", "stddev": "9.35", "min": "0.0", "max": "30.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Weight"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Volume", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "43022", "mean": "894.59", "stddev": "2176.45", "min": "2", "max": "32340", "missing": "0"}, "updatedLabel": "Volume"}, {"transformationsData": [{"feature_label": "CarrierPickedUpDateTime", "transformation_label": "String Indexer"}], "feature": "CarrierPickedUpDateTime", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "2020-01-07 21:17", "max": "2021-02-09 07:21", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CarrierPickedUpDateTime"}, {"transformationsData": [{"feature_label": "CarrierDeliveredDateTime", "transformation_label": "String Indexer"}], "feature": "CarrierDeliveredDateTime", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "43022", "mean": "", "stddev": "", "min": "2020-01-05 01:18", "max": "2021-03-09 07:21", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CarrierDeliveredDateTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShipCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "43022", "mean": "6.38", "stddev": "2.75", "min": "1.0", "max": "12.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ShipCost"}, {"feature": "Carrier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "43022", "mean": "0.99", "stddev": "0.82", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Carrier_stringindexer_tra..."}, {"feature": "ProductName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "43022", "mean": "134.36", "stddev": "110.95", "min": "0.0", "max": "355.0", "missing": "0"}, "updatedLabel": "ProductName_stringindexer..."}, {"feature": "ToZip_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "43022", "mean": "143.97", "stddev": "117.17", "min": "0.0", "max": "371.0", "missing": "0"}, "updatedLabel": "ToZip_stringindexer_trans..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 1309211Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=randomforestregressor(autofe, ["ShippingId", "Carrier_stringindexer", "ProductName_stringindexer", "Qty", "Length", "ToZip_stringindexer", "quantity", "Weight", "Volume"], "ShipCost")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
